In [4]:
import csv
import serial
import json
import sqlite3
from datetime import datetime, timedelta
import serial.tools.list_ports
from datetime import datetime
import paho.mqtt.client as mqtt

In [1]:
import sqlite3
import pandas as pd

# 连接到数据库
db_file = "sensor_data.db"
conn = sqlite3.connect(db_file)

# 将数据加载到 DataFrame
query = "SELECT * FROM sensor_data"
df = pd.read_sql_query(query, conn)

# 打印 DataFrame
print(df)

# 关闭连接
conn.close()

      id            timestamp    DO     TDS      Tur     pH    Temp
0      1  2025-03-25 10:42:17  0.14  391.73  2986.44   9.69   96.13
1      2  2025-03-25 10:42:26  0.06  386.93  2986.44   9.69   97.58
2      3  2025-03-25 10:42:36  0.06  386.93  2985.01   9.69   97.58
3      4  2025-03-25 10:42:46 -0.22  382.24  2985.01   9.69   99.04
4      5  2025-03-25 10:42:55 -0.22  382.98  2983.53   9.72   99.04
..   ...                  ...   ...     ...      ...    ...     ...
138  139  2025-03-25 11:04:21 -0.11  366.46  2957.22  10.05  107.93
139  140  2025-03-25 11:04:30 -0.11  366.46  2957.22  10.05  107.93
140  141  2025-03-25 11:04:40  0.00  362.30  2957.22  10.05  109.44
141  142  2025-03-25 11:04:49  0.00  370.72  2957.22  10.05  106.43
142  143  2025-03-25 11:04:59  0.00  370.72  2957.22  10.05  106.43

[143 rows x 7 columns]


In [ ]:
broker_address = "127.0.0.1"  
broker_port = 1883
send_topic = "AQ/data"  
request_topic = "AQ/request"  
response_topic = "AQ/response"  

client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)

# 连接到MQTT Broker 
client.connect(broker_address, broker_port)
client.subscribe(send_topic)  
client.subscribe(response_topic)  
client.loop_start()

<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [ ]:

def get_user_input():
    while True:
        try:
            start_time = input("Enter start time (YYYY-MM-DD HH:MM:SS): ")
            end_time = input("Enter end time (YYYY-MM-DD HH:MM:SS): ")

            # 验证时间格式
            datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S")
            datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S")

            return start_time, end_time
        except ValueError:
            print("Invalid time format. Please use YYYY-MM-DD HH:MM:SS.")

def request_history(start_time, end_time):
    payload = json.dumps({"start_time": start_time, "end_time": end_time})
    client.publish(request_topic, payload)
    print(f"Sent request for data from {start_time} to {end_time}")

def query_history(start_time, end_time):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    cursor.execute("SELECT timestamp, DO, TDS, Tur, pH, Temp FROM sensor_data WHERE timestamp BETWEEN ? AND ?", (start_time, end_time))
    data = cursor.fetchall()
    conn.close()
    return data

db_file = "sensor_data.db"

#start_time, end_time = get_user_input()
start_time = "2025-03-20 21:57:59"
end_time = "2025-03-20 21:58:09"
request_history(start_time, end_time)
history_data = query_history(start_time, end_time)
response = json.dumps(history_data)

# 发送历史数据到接收端
client.publish(response_topic, response)
print(f"Sent history data to {response_topic}")

Sent request for data from 2025-03-20 21:57:59 to 2025-03-20 21:58:09
Sent history data to AQ/response
